In [1]:
%matplotlib inline
import itertools
import os
import numpy as np
import gpflow
import numbers
import matplotlib.pyplot as plt
import tensorflow as tf
from gpflow_monitor import *
X = np.random.rand(10000, 1) * 10
Y = np.sin(X) + np.random.randn(*X.shape)
Xt = np.random.rand(10000, 1) * 10
Yt = np.sin(X) + np.random.randn(*X.shape)
np.random.seed(0)

# Demo: `gpflow_monitor`
In this notebook we'll demo how to use `gpflow_monitoring` for logging the optimisation of a GPflow model. The example should cover pretty much all use cases.

## Creating the GPflow model
We first create the GPflow model as usual.

In [2]:
m = gpflow.models.SVGP(X, Y, gpflow.kernels.RBF(1), gpflow.likelihoods.Gaussian(), Z=np.linspace(0, 10, 5)[:, None],
                       minibatch_size=100)
m.likelihood.variance = 0.01
m.compile()

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.


## Setting up the optimisation
Next we need to set up the optimisation process. `gpflow_monitor` provides classes that manage the optimsation, and perform certain logging tasks. In this example, we want to:
- log certain scalar parameters in TensorBoard
- log the full optimisation objective (log marginal likelihood bound) periodically, even though we optimise with minibatches
- store a backup of the optimisation process periodically
- log performance for a test set periodically

Because of the integration with TensorFlow ways of storing and logging, we will need to perform a few TensorFlow manipulations outside of GPflow as well.

We start by creating the `global_step` variable. This is not strictly required by TensorFlow optimisers, but they do all have support for it. Its purpose is to track how many optimisation steps have occurred. It is useful to keep this in a TensorFlow variable as this allows it to be restored together with all the parameters of the model.

In [3]:
global_step = tf.Variable(0, trainable=False, name="global_step")
m.session.run(global_step.initializer)

Next, we create an instance of `FileWriter`, which will save the TensorBoard logs to a file. This object needs to be shared between all `gpflow_monitor.TensorBoard` objects, if they are to write to the same path.

In [4]:
fw = tf.summary.FileWriter(os.path.join("./results/test/tensorboard/"), m.session.graph)

Now the TensorFlow side is set up, we can focus on the `gpflow_monitor` part. The optimsation is taken care of by the `ManagedOptimisation` class. This will run the training loop. The `ManagedOptimisation` object will also take care of running `Task`s.

Each `Task` is something that needs to be run periodically during the optimisation. The first and second parameters of all tasks are a generator returning times (either in iterations or time) of when the `Task` needs to be run. The second determines whether a number of iterations (`Trigger.ITER`), an amount of time spent optimising (`Trigger.OPTIMISATION_TIME`), or the wall-clock time (`Trigger.TOTAL_TIME`) triggers the `Task` to be run. The following `Task`s are run once in every 100 or 1000 iterations.

In [5]:
opt_method = ManagedOptimisation(m, gpflow.train.AdamOptimizer(0.01), global_step)
opt_method.tasks += [
    PrintTimings((x * 100 for x in itertools.count()), Trigger.ITER),
    ModelTensorBoard((x * 100 for x in itertools.count()), Trigger.ITER, m, fw),
    LmlTensorBoard((x * 1000 for x in itertools.count()), Trigger.ITER, m, fw, verbose=False),
    StoreSession((x * 1000 for x in itertools.count()), Trigger.ITER, m.session, "./results/test/checkpoint")
]

INFO:tensorflow:Summary name full lml is illegal; using full_lml instead.
Restoring session from `./results/test/checkpoint-8000`.
INFO:tensorflow:Restoring parameters from ./results/test/checkpoint-8000


We may also want to perfom certain tasks that do not have pre-defined `Task` classes. For example, computing the performance on a test set. Here we create such a class by extending `ModelTensorBoard` to log the testing benchmarks in addition to all the scalar parameters.

In [6]:
class TestTensorBoard(ModelTensorBoard):
    def __init__(self, sequence, trigger: Trigger, model, file_writer, Xt, Yt):
        super().__init__(sequence, trigger, model, file_writer)
        self.Xt = Xt
        self.Yt = Yt
        self._full_test_err = tf.placeholder(gpflow.settings.tf_float, shape=())
        self._full_test_nlpp = tf.placeholder(gpflow.settings.tf_float, shape=())

        self.summary = tf.summary.merge([tf.summary.scalar("test_rmse", self._full_test_err),
                                         tf.summary.scalar("test_nlpp", self._full_test_nlpp)])

    def _event_handler(self, manager):
        minibatch_size = 100
        preds = np.vstack([m.predict_y(Xt[mb * minibatch_size:(mb + 1) * minibatch_size, :])[0]
                            for mb in range(-(-len(Xt) // minibatch_size))])
        test_err = np.mean((Yt - preds) ** 2.0)**0.5
        summary, step = m.session.run([self.summary, global_step],
                                      feed_dict={self._full_test_err: test_err,
                                                 self._full_test_nlpp: 0.0})
        self.file_writer.add_summary(summary, step)

We then add it to the task list.

In [7]:
opt_method.tasks.append(TestTensorBoard((x * 1000 for x in itertools.count()), Trigger.ITER, m, fw, Xt, Yt))

## Running the optimisation
We finally get to running the optimisation. The second time this is run, the session should be restored from a checkpoint created by `StoreSession`. To confirm this, we print out the first value in all TensorFlow tensors. This includes any values used by the optimiser. This is important to ensure that the optimiser starts off from _exactly_ the same state as that it left. If this is not done correctly, models may start diverging after loading.

In [8]:
[u[1] if isinstance(u[1], numbers.Number) else u[1].flatten()[0]  for u in sorted([(v.name, m.session.run(v)) for v in tf.global_variables()], key=lambda x: x[0])]

[-29.671438513350214,
 105642.70244418224,
 0.60462358561365404,
 177.0548159669888,
 572052.05230759573,
 2.2795021624035749,
 -13.968892500214993,
 521734.970636243,
 0.74429984134674099,
 23.349998194087412,
 17994155.75995528,
 0.54319964666710507,
 66.140432611715582,
 407294.53293196583,
 0.52595946097680712,
 3.1432234523936655,
 76466.479372075162,
 0.024963534604432597,
 0.0,
 0.0003338237,
 8000]

In [9]:
opt_method.minimize(m, maxiter=8000)

1, 8001:	3.00 optimisation iter/s	3.00 total iter/s	0.00 last iter/sFull lml: -14181.634753 (-1.42e+04)
1000, 9000:	761.37 optimisation iter/s	547.18 total iter/s	995.63 last iter/sFull lml: -14166.445147 (-1.42e+04)
2000, 10000:	850.46 optimisation iter/s	640.62 total iter/s	921.44 last iter/sFull lml: -14160.863033 (-1.42e+04)
3000, 11000:	893.53 optimisation iter/s	683.87 total iter/s	996.39 last iter/sFull lml: -14173.089808 (-1.42e+04)
4000, 12000:	911.25 optimisation iter/s	704.14 total iter/s	1010.41 last iter/sFull lml: -14196.183877 (-1.42e+04)
5000, 13000:	931.59 optimisation iter/s	721.71 total iter/s	1014.06 last iter/sFull lml: -14180.076029 (-1.42e+04)
6000, 14000:	944.22 optimisation iter/s	733.91 total iter/s	998.69 last iter/sFull lml: -14159.440618 (-1.42e+04)
7000, 15000:	949.26 optimisation iter/s	740.49 total iter/s	966.56 last iter/sFull lml: -14163.653298 (-1.42e+04)
8000, 16000:	953.53 optimisation iter/s	745.74 total iter/s	958.96 last iter/sFull lml: -14164.27

Here, we print the optimised variables for comparison on the next run.

In [10]:
[u[1] if isinstance(u[1], numbers.Number) else u[1].flatten()[0]  for u in sorted([(v.name, m.session.run(v)) for v in tf.global_variables()], key=lambda x: x[0])]

[-29.671438513350214,
 105642.70244418224,
 0.60462358561365404,
 177.0548159669888,
 572052.05230759573,
 2.2795021624035749,
 -13.968892500214993,
 521734.970636243,
 0.74429984134674099,
 23.349998194087412,
 17994155.75995528,
 0.54319964666710507,
 66.140432611715582,
 407294.53293196583,
 0.52595946097680712,
 3.1432234523936655,
 76466.479372075162,
 0.024963534604432597,
 0.0,
 0.0003338237,
 8000]